In [3]:
import pandas as pd
from ast import literal_eval
import os
import time
from tqdm import tqdm


# === CONFIGURATION ===
GEMINI_CSV = "psychforums_dataset_cleaned_final_1.csv"
COHERE_CSV = "psychforums_dataset_cleaned_final_2.csv"
BATCH_SIZE = 2000
RESUME_FILE = "resolved_psychforums_dataset.csv"
ID_COLUMN = "id"
TEXT_COLUMN = "text"
LABEL_COLUMN = "label"

# === LOAD BOTH FILES ===
df_gemini = pd.read_csv(GEMINI_CSV)
df_cohere = pd.read_csv(COHERE_CSV)

# Extract only id, text, and labels as list for Gemini
label_cols = ['label_0', 'label_1', 'label_2', 'label_3']
df_gemini['labels_gemini'] = df_gemini[label_cols].values.tolist()

# Extract only id and labels as list for Cohere
df_cohere['labels_cohere'] = df_cohere[label_cols].values.tolist()

# Merge — keep text from Gemini file
df_merged = pd.merge(
    df_gemini[[ID_COLUMN, TEXT_COLUMN, 'labels_gemini']],
    df_cohere[[ID_COLUMN, 'labels_cohere']],
    on=ID_COLUMN
)

In [4]:
def parse_label(label):
    try:
        if isinstance(label, str):
            return literal_eval(label)
        elif isinstance(label, (int, float)):
            return [int(label)]
        elif isinstance(label, list):
            return [int(x) for x in label]
    except Exception:
        pass
    return []

# Load already resolved data if exists
if os.path.exists(RESUME_FILE):
    df_resolved = pd.read_csv(RESUME_FILE)
    resolved_ids = set(df_resolved[ID_COLUMN])
    resolved_rows = df_resolved.to_dict(orient="records")
    print(f"🔁 Resuming from previous session: {len(resolved_ids)} already resolved.\n")
else:
    resolved_rows = []
    resolved_ids = set()

🔁 Resuming from previous session: 1645 already resolved.



In [5]:
## --- Main loop ---
disagreements_handled = 0

for _, row in tqdm(df_merged.iterrows(), total=len(df_merged)):
    id_ = row[ID_COLUMN]
    
    if id_ in resolved_ids:
        continue

    labels_gemini = row['labels_gemini']
    labels_cohere = row['labels_cohere']
    text = row[TEXT_COLUMN]

    if labels_gemini == labels_cohere:
        resolved_label_list = labels_gemini
        resolved_ids.add(id_)
    else:
        print(f"\n--- Disagreement for ID {id_} ---")
        print(f"Text: {row[TEXT_COLUMN]}")
        print(f"Gemini Label: {row['labels_gemini']}")
        print(f"Cohere Label: {row['labels_cohere']}")
        while True:
            try:
                user_input = input("Enter correct label(s) (e.g., 0 2), or 's' to skip batch: ").strip()
                if user_input.lower() == 's':
                    print("🛑 Batch stopped by user.")
                    break
                res = [int(x) for x in user_input.split() if int(x) in [0, 1, 2, 3]]
                if res:
                    resolved_label_list = [1 if i in res else 0 for i in range(4)]
                    break
                else:
                    print("Empty or invalid input. Try again.")
            except ValueError:
                print("Invalid input. Please enter space-separated numbers from 0 to 3.")

    resolved_rows.append({
        ID_COLUMN: id_,
        TEXT_COLUMN: row[TEXT_COLUMN],
        "label_0": resolved_label_list[0],
        "label_1": resolved_label_list[1],
        "label_2": resolved_label_list[2],
        "label_3": resolved_label_list[3]
    })
    resolved_ids.add(id_)
        
    disagreements_handled += 1
    print(disagreements_handled)

    if disagreements_handled >= BATCH_SIZE:
        print(f"\n✅ Reached batch limit of {BATCH_SIZE}. Saving progress.")
        break

  0%|                                                  | 0/2599 [00:00<?, ?it/s]


--- Disagreement for ID 1646 ---
Text: paranoia? funny i get something like this sometimes, but i think it's somehow insecurity based. i remember reading about the self-insecure personality: "according to jaspers, such people experience inner humiliation, brought about by outside experiences and their interpretations of them. they have an urge to get external confirmation to their self-deprecation and that makes them see insults in the behavior of other people. they suffer from every slight because they seek the real reason for them in themselves." basically that. go and get some vape juice, guy sells me nicotine-free ones, presume it's because i supposedly look younger than my age then feel insulted. someone disposes on some food i was going to have in front of me, "oh did you want that?", think they're trying to get under my skin by slighting something i wanted. "do you want help with that?" obviously must imply (at the time) that i'm incapable or doing something. self-centered thou

Enter correct label(s) (e.g., 0 2), or 's' to skip batch:  2


 63%|████████████████████████▋              | 1646/2599 [00:22<00:13, 71.70it/s]

1

--- Disagreement for ID 1647 ---
Text: paranoia? yes, i can definitely identify with this problem. i just read the "paranoid phenomena and pathological narcissism" link and this sentence particularly resonated with me: "tragically, being gripped by the paranoid condition and its manifest delusional system is the only kind of security that the paranoid patient knows." i would indeed feel lost without my paranoia because i have become so accustomed to it as a form of self-defence and self-preservation. i was bullied and ostracized at school and i became paranoid and defensive in response to the repeated rejections from my peers. i would trust people were my friends and then hear from other people they were being two-faced and didn't really like me etc. it feels much safer to assume the worst in everybody in case i am let down yet again. this is why i also find myself very uncomfortable when somebody appears to genuinely take a liking to me because, for one, i don't believe i am worthy

...

100%|██████████████████████████████████████▉| 2597/2599 [35:49<00:02,  1.43s/it]

952

--- Disagreement for ID 2598 ---
Text: dear faithful, my mother died a few months ago after a long decline from parkinson's. she was similar to your dad in that she eventually could barely speak, just make little grunts, so it was really hard to know what she was thinking. her facial expression was also rigid, so reading emotional content was impossible. we tried as much as possible to relate to her as though she understood us. it was sort of like having a baby, but without the expressiveness infants show. very difficult. my heart goes out to you. if he still can recognize you, i would think going to see him alot, and providing him with familiar objects, like a favorite blanket, ect. might help him cope. i often felt that my mother was lost in a big house she used to know her way around in. she would look out the windows so to speak, but not recognize the neighborhood. very lonely. so like i said, go see him alot. go sit with him and don't worry too much about feedback. just pet h

Enter correct label(s) (e.g., 0 2), or 's' to skip batch:  1


100%|███████████████████████████████████████| 2599/2599 [35:50<00:00,  1.21it/s]

953
954


In [6]:
# --- Save progress ---
pd.DataFrame(resolved_rows).to_csv(RESUME_FILE, index=False)
print(f"💾 Saved {len(resolved_rows)} new resolved entries to {RESUME_FILE}")

💾 Saved 2599 new resolved entries to resolved_psychforums_dataset.csv
